## importing the model

In [48]:
from cobra.io import read_sbml_model, write_sbml_model
model = read_sbml_model('/Users/wagner/github-classroom/27410/group-assingment-team-4/yeast-GEM-8.7.0/model/yeast-GEM2.xml') # loading
from cobra import Reaction, Metabolite, Gene
import pandas as pd



# Finding the most suitable carbon source and testing different types of media

First we can test the production of beta-farnesene in aerobic conditions as well as anaerobic conditions:

In [49]:
model.reactions.query('oxygen exchange', 'name')

[<Reaction r_1992 at 0x17eacff10>]

In [50]:
with model:
   medium = model.medium
   medium['r_1992'] = 0
   model.medium = medium
   with model:
      model.objective = model.reactions.BFD
      BF_production = model.optimize().objective_value
print(BF_production)

0.04739229024943303


There is no production in this case no point in looking further at that, so we will go back to aerobic conditions:

In [52]:
with model:
    growth_minimal_medium = model.optimize().objective_value
    print('Growth rate on minimal media:', growth_minimal_medium, 'gDW h^-1')
    model.objective = model.reactions.BFD
    BF_production_minimal_medium = model.optimize().objective_value
    print("Maximum beta-farnesene production on minimal media [mmol-beta-farnesene gDW^-1 h^-1]:", BF_production_minimal_medium)
    print("Theoretical maximum yield on minimal media [mmol-beta-farnesene / mmol-glucose]:", BF_production_minimal_medium / (-1*model.reactions.r_1714.flux))
    max_theoretical_yield_minimal_medium = BF_production / (-1*model.reactions.r_1714.flux)

Growth rate on minimal media: 0.08192815852363902 gDW h^-1
Maximum beta-farnesene production on minimal media [mmol-beta-farnesene gDW^-1 h^-1]: 0.23035112219719808
Theoretical maximum yield on minimal media [mmol-beta-farnesene / mmol-glucose]: 0.23035112219719808


Now we will test to see the production of beta-farnesene with increased glucose concentrations:

In [5]:
# First we will here make an overview of the full medium composition as it is in the model by default with
# growth on glucose, to do this we check through all the extracellular exchange reactions:

for component in model.medium.items():
    print(model.reactions.get_by_id(component[0]), "|", model.reactions.get_by_id(component[0]).name, "| concentration =", component[1])

r_1654: s_0420 <=>  | ammonium exchange | concentration = 1000.0
r_1714: s_0565 <=>  | D-glucose exchange | concentration = 10.0
r_1832: s_0796 <=>  | H+ exchange | concentration = 1000.0
r_1861: s_0925 <=>  | iron(2+) exchange | concentration = 1000.0
r_1992: s_1277 <=>  | oxygen exchange | concentration = 1000.0
r_2005: s_1324 <=>  | phosphate exchange | concentration = 1000.0
r_2020: s_1374 <=>  | potassium exchange | concentration = 1000.0
r_2049: s_1438 <=>  | sodium exchange | concentration = 1000.0
r_2060: s_1468 <=>  | sulphate exchange | concentration = 1000.0
r_2100: s_0805 <=>  | water exchange | concentration = 1000.0
r_4593: s_4200 <=>  | chloride exchange | concentration = 1000.0
r_4594: s_4201 <=>  | Cu2(+) exchange | concentration = 1000.0
r_4595: s_4202 <=>  | Mn(2+) exchange | concentration = 1000.0
r_4596: s_4203 <=>  | Zn(2+) exchange | concentration = 1000.0
r_4597: s_4204 <=>  | Mg(2+) exchange | concentration = 1000.0
r_4600: s_4199 <=>  | Ca(2+) exchange | conce

This is a full overview of the medium composition. Using this we will now check the impact of changing the concentration of glucose in the medium on production and on growth:

In [40]:
# Testing different glucose concentrations of 0.1X, 10X, 100X and 1000X the standard glucose concentration in the medium

medium = model.medium
with model:
   medium['r_1714'] = 0.1
   model.medium = medium
   point_one_X_glc_growth = model.optimize().objective_value
   model.objective = model.reactions.BFD
   point_one_X_glc_productivity = model.optimize().objective_value
   print('Maximal growth with 0.1X glucose:', point_one_X_glc_growth)
   print('Maximal production of beta-farnesene with 0.1X glucose:', point_one_X_glc_productivity)
   print('Theoretical maximum beta-farnesene production on 0.1X glucose:', model.optimize().objective_value / (-1*model.reactions.r_1714.flux))

print('')

with model:
   medium['r_1714'] = 1
   model.medium = medium
   one_X_glc_growth = model.optimize().objective_value
   model.objective = model.reactions.BFD
   one_X_glc_productivity = model.optimize().objective_value
   print('Maximal growth with 1X glucose:', one_X_glc_growth)
   print('Maximal production of beta-farnesene with 1X glucose:', one_X_glc_productivity)
   print('Theoretical maximum beta-farnesene production on 1X glucose:', model.optimize().objective_value / (-1*model.reactions.r_1714.flux))


print('')


with model:
   medium['r_1714'] = 10
   model.medium = medium
   ten_X_glc_growth = model.optimize().objective_value
   model.objective = model.reactions.BFD
   ten_X_glc_productivity = model.optimize().objective_value
   print('Maximal growth with 10X glucose:', ten_X_glc_growth)
   print('Maximal production of beta-farnesene with 10X glucose:', ten_X_glc_productivity)
   print('Theoretical maximum beta-farnesene production on 10X glucose:', model.optimize().objective_value / (-1*model.reactions.r_1714.flux))

print('')

with model:
   medium['r_1714'] = 100
   model.medium = medium
   hundred_X_glc_growth = model.optimize().objective_value
   model.objective = model.reactions.BFD
   hundred_X_glc_productivity = model.optimize().objective_value
   print('Maximal growth with 100X glucose:', hundred_X_glc_growth)
   print('Maximal production of beta-farnesene with 100X glucose:', hundred_X_glc_productivity)
   print('Theoretical maximum beta-farnesene production on 100X glucose:', model.optimize().objective_value / (-1*model.reactions.r_1714.flux))

print('')

with model:
   medium['r_1714'] = 1000
   model.medium = medium
   thousand_X_glc_growth = model.optimize().objective_value
   model.objective = model.reactions.BFD
   thousand_X_glc_productivity = model.optimize().objective_value
   print('Maximal growth with 1000X glucose:', thousand_X_glc_growth)
   print('Maximal production of beta-farnesene with 1000X glucose:', thousand_X_glc_productivity)
   print('Theoretical maximum beta-farnesene production on 1000X glucose:', model.optimize().objective_value / (-1*model.reactions.r_1714.flux))



   

Maximal growth with 0.1X glucose: 0.005401619769345875
Maximal production of beta-farnesene with 0.1X glucose: 0.016662111190072968
Theoretical maximum beta-farnesene production on 0.1X glucose: 0.16662111190072693

Maximal growth with 1X glucose: 0.08192813815902517
Maximal production of beta-farnesene with 1X glucose: 0.23035112219719797
Theoretical maximum beta-farnesene production on 1X glucose: 0.23035112219719536

Maximal growth with 10X glucose: 0.847193021894946
Maximal production of beta-farnesene with 10X glucose: 2.3416532799479004
Theoretical maximum beta-farnesene production on 10X glucose: 0.23416532799479045

Maximal growth with 100X glucose: 8.499842132127739
Maximal production of beta-farnesene with 100X glucose: 23.45467485745477
Theoretical maximum beta-farnesene production on 100X glucose: 0.23454674857455005

Maximal growth with 1000X glucose: 19.817859616711658
Maximal production of beta-farnesene with 1000X glucose: 76.91806853582565
Theoretical maximum beta-farn

It seems that there is little to no benefit in the theoretical maximum of beta-farnesene production neither increasing nor decreasing the glucose concentrations.


# Changing to a medium composition that mimicks YPD medium

For further medium optimization we will now look into changing the medium composition from the standard minimal medium of the model to the complete medium for yeast growth YPD medium also known as YPED medium. 
The composition of the mimicking media has been obtained by using a glucose uptake of 5 mmol gDW<sup>-1</sup> h<sup>-1</sup> and mimicking a 1% concentration of yeast extract with concentrations found from [1] and adjusted accordingly.

In [67]:
YPD_components = ['Glucose', 'Alanine', 'Arginine', 'Aspartic', 'Cysteine', 'Glutamine', 'Glycine', 'Histidine', 'Isoleucine',
                  'Leucine', 'Lysine', 'Methionine', 'Phenylalanine', 'Proline', 'Serine', 'Threonine', 'Tryptophan',
                  'Tyrosine', 'Valine', 'Zinc', 'FE2']

YPD_medium = model.medium

YPD_medium['r_1714'] = 5
YPD_medium['r_1873'] = 0.44
YPD_medium['r_1879'] = 0.25
YPD_medium['r_1881'] = 0.49
YPD_medium['r_1883'] = 0.04
YPD_medium['r_1891'] = 0.81
YPD_medium['r_1810'] = 0.24
YPD_medium['r_1893'] = 0.10
YPD_medium['r_1897'] = 0.27
YPD_medium['r_1899'] = 0.38
YPD_medium['r_1900'] = 0.40
YPD_medium['r_1902'] = 0.09
YPD_medium['r_1903'] = 0.23
YPD_medium['r_1904'] = 0.20
YPD_medium['r_1906'] = 0.23
YPD_medium['r_1911'] = 0.21
YPD_medium['r_1912'] = 0.14
YPD_medium['r_1913'] = 0.06
YPD_medium['r_1914'] = 0.29
YPD_medium['r_4596'] = 0.00056
YPD_medium['r_1861'] = 0.0005

model.objective = model.reactions.r_2111
model.medium = YPD_medium
with model:
      YPD_growth = model.optimize().objective_value
      model.objective = model.reactions.BFD
      YPD_productivity = model.optimize().objective_value
      print('Maximal growth with YPD media:', YPD_growth, 'gDW h^-1')
      print('Maximal theoretical production of beta-farnesene with YPD media:', YPD_productivity, 'mmol gDW^-1 h^-1')
      print('Theoretical maximum yield of beta-farnesene on YPD media:', 
            model.optimize().objective_value / (-1*model.reactions.r_1714.flux), 'mmol-BF mmol-glc^-1')
   

Maximal growth with YPD media: 0.7366856920674737 gDW h^-1
Maximal theoretical production of beta-farnesene with YPD media: 1.6073361629778653 mmol gDW^-1 h^-1
Theoretical maximum yield of beta-farnesene on YPD media: 0.3214672325955719 mmol-BF mmol-glc^-1


# Testing different carbon sources for optimal production

In [68]:
model_medium = model.medium
productivity = []
reaction_name = []
theoretical_yield = []
theoretical_yield_c_mol = []
theoretical_yield_c_mol_conversion = [5, 6, 6, 6, 6, 5, 5, 5, 4, 6, 12, 12, 18, 12, 12, 6, 12, 6, 18, 6, 6, 6]

with model:
   for reac in model.exchanges.query('ose','name'):
      medium = model_medium
      medium['r_1714'] = 0
      medium[reac.id] = 5
      model.medium = medium
      model.objective = model.reactions.BFD
      productivity.append(model.optimize().objective_value)
      reaction_name.append(reac.name)
      if reac.flux != 0:
         theoretical_yield.append(model.optimize().objective_value / (-1 * reac.flux))
         theoretical_yield_c_mol.append((model.optimize().objective_value * 15 / (-1 * reac.flux*theoretical_yield_c_mol_conversion[model.exchanges.query('ose','name').index(reac)-1])))
      else:
         theoretical_yield.append(0)
         theoretical_yield_c_mol.append(0)
      medium[reac.id] = 0
      model.medium = medium




In [69]:
productivity_on_carbon_sources = pd.DataFrame({'Carbon source': reaction_name,
   'Theoretical productivity of Beta-farnesene': productivity, 'Theoretical yield': theoretical_yield,
   'Theoretical yield in cmol':theoretical_yield_c_mol})
print(productivity_on_carbon_sources.sort_values(by='Theoretical yield in cmol', ascending = False).head(15).to_markdown())


|    | Carbon source                  |   Theoretical productivity of Beta-farnesene |   Theoretical yield |   Theoretical yield in cmol |
|---:|:-------------------------------|---------------------------------------------:|--------------------:|----------------------------:|
|  0 | trehalose exchange             |                                      2.78051 |            0.556101 |                    1.39025  |
|  6 | D-ribose exchange              |                                      1.41019 |            0.282038 |                    0.846115 |
|  9 | L-homoserine exchange          |                                      1.12377 |            0.224755 |                    0.84283  |
|  1 | D-arabinose exchange           |                                      1.37477 |            0.274955 |                    0.824865 |
|  7 | D-xylose exchange              |                                      1.37477 |            0.274955 |                    0.824865 |
| 18 | D-glucose 6-phosphat

In [70]:
print(productivity_on_carbon_sources.sort_values(by='Theoretical productivity of Beta-farnesene', ascending = False).head(15).to_markdown())

|    | Carbon source                                        |   Theoretical productivity of Beta-farnesene |   Theoretical yield |   Theoretical yield in cmol |
|---:|:-----------------------------------------------------|---------------------------------------------:|--------------------:|----------------------------:|
| 19 | alpha-maltotriose exchange                           |                                      3.95854 |            0.791709 |                    0.659757 |
| 12 | sucrose exchange                                     |                                      2.78784 |            0.557569 |                    0.696961 |
| 17 | turanose exchange                                    |                                      2.78784 |            0.557569 |                    0.696961 |
|  0 | trehalose exchange                                   |                                      2.78051 |            0.556101 |                    1.39025  |
| 11 | maltose exchange           

In [48]:
for reac in model.exchanges.query('ose','name'):
   print(reac.name)
   print(reac.id)

trehalose exchange
r_1650
D-arabinose exchange
r_1706
D-fructose exchange
r_1709
D-galactose exchange
r_1710
D-glucose exchange
r_1714
D-mannose exchange
r_1715
D-ribose exchange
r_1716
D-xylose exchange
r_1718
L-arabinose exchange
r_1878
L-homoserine exchange
r_1896
L-sorbose exchange
r_1909
maltose exchange
r_1931
sucrose exchange
r_2058
raffinose exchange
r_4043
melibiose exchange
r_4044
6-O-alpha-D-glucopyranosyl-D-fructofuranose exchange
r_4498
D-tagatose exchange
r_4500
turanose exchange
r_4501
D-glucose 6-phosphate exchange
r_4502
alpha-maltotriose exchange
r_4503
D-glucose 1-phosphate exchange
r_4504
D-mannose 6-phosphate exchange
r_4539
D-mannose 1-phosphate exchange
r_4547


# Saving the model with new medium composition (YPD medium)

In [26]:
# This code saves the changes to the model and we now have a model that has a the heterologous pathway for 
# beta-farnesene production as well as a demand reaction for cytoplasmic beta-farnesene

write_sbml_model(model,
                 filename = '/Users/wagner/github-classroom/27410/group-assingment-team-4/yeast-GEM-8.7.0/model/yeast-GEM3.xml'
                 )  
#saving the model

# references

[1] Yeast Extracts: Nutritional and Flavoring Food Ingredients, <i>CS Food Sci. Technol</di>. 2021, 1, 4, 487–494
Publication Date:April 28, 2021
https://doi.org/10.1021/acsfoodscitech.0c00131